In [1]:
# 1. SETUP AND DATA DOWNLOAD

# Core Python + data handling
import os
import glob
import re
import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1a. Download OpenI / IU chest X-ray reports (XML only, no images)
# The archive contains ecgen-radiology XML files with FINDINGS/IMPRESSION sections.
!wget -q https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz

# 1b. Extract the .tgz archive
!tar -xvzf NLMCXR_reports.tgz

# Quick check of the extracted contents (should include 'ecgen-radiology/')
!ls


ecgen-radiology/
ecgen-radiology/1.xml
ecgen-radiology/10.xml
ecgen-radiology/100.xml
ecgen-radiology/1000.xml
ecgen-radiology/1001.xml
ecgen-radiology/1002.xml
ecgen-radiology/1003.xml
ecgen-radiology/1004.xml
ecgen-radiology/1005.xml
ecgen-radiology/1006.xml
ecgen-radiology/1007.xml
ecgen-radiology/1008.xml
ecgen-radiology/1009.xml
ecgen-radiology/101.xml
ecgen-radiology/1010.xml
ecgen-radiology/1011.xml
ecgen-radiology/1012.xml
ecgen-radiology/1013.xml
ecgen-radiology/1014.xml
ecgen-radiology/1015.xml
ecgen-radiology/1016.xml
ecgen-radiology/1017.xml
ecgen-radiology/1018.xml
ecgen-radiology/1019.xml
ecgen-radiology/102.xml
ecgen-radiology/1020.xml
ecgen-radiology/1021.xml
ecgen-radiology/1022.xml
ecgen-radiology/1023.xml
ecgen-radiology/1024.xml
ecgen-radiology/1025.xml
ecgen-radiology/1026.xml
ecgen-radiology/1027.xml
ecgen-radiology/1028.xml
ecgen-radiology/1029.xml
ecgen-radiology/103.xml
ecgen-radiology/1030.xml
ecgen-radiology/1031.xml
ecgen-radiology/1032.xml
ecgen-radiology/1

In [2]:
# 2. LOCATE XML REPORT FILES

# All chest X-ray reports are stored as XML files in the ecgen-radiology folder.
reports_root = "ecgen-radiology"

# Recursively find all XML report files
xml_files = glob.glob(os.path.join(reports_root, "**", "*.xml"), recursive=True)

print("Number of XML report files found:", len(xml_files))
xml_files[:10]  # preview a few paths


Number of XML report files found: 3955


['ecgen-radiology/401.xml',
 'ecgen-radiology/3738.xml',
 'ecgen-radiology/3099.xml',
 'ecgen-radiology/862.xml',
 'ecgen-radiology/3834.xml',
 'ecgen-radiology/3642.xml',
 'ecgen-radiology/322.xml',
 'ecgen-radiology/3565.xml',
 'ecgen-radiology/1101.xml',
 'ecgen-radiology/683.xml']

In [3]:
# 3. PARSE XML TO EXTRACT REPORT TEXT

def extract_report_text(xml_path):
    """
    Parse a single OpenI / IU chest X-ray XML report and extract
    the FINDINGS and IMPRESSION sections.

    Returns:
        str: Combined findings + impression text for that report.
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
    except Exception:
        # If parsing fails for any reason, skip this report
        return ""

    findings = []
    impression = []

    # Report text is usually stored in <AbstractText Label="FINDINGS">, etc.
    for abs_text in root.iter("AbstractText"):
        label = abs_text.attrib.get("Label", "").lower()
        # Join all text fragments inside this AbstractText node
        text = "".join(abs_text.itertext()).strip()

        if not text:
            continue

        if "finding" in label:
            findings.append(text)
        elif "impression" in label:
            impression.append(text)

    # Fallback: if there are no explicit FINDINGS/IMPRESSION labels,
    # join all AbstractText contents as a single report_text.
    if not findings and not impression:
        all_text = [
            "".join(a.itertext()).strip()
            for a in root.iter("AbstractText")
            if "".join(a.itertext()).strip()
        ]
        return " ".join(all_text)

    # Normal case: combine findings and impression into one string
    return " ".join(findings + impression).strip()


In [4]:
# 4. BUILD DATAFRAME OF RADIOLOGY REPORTS

rows = []

for path in xml_files:
    # Use the XML filename (without .xml) as a simple report_id
    report_id = os.path.splitext(os.path.basename(path))[0]
    text = extract_report_text(path)

    rows.append({
        "report_id": report_id,
        "report_text": text
    })

df = pd.DataFrame(rows)

print("Raw df shape (before cleaning):", df.shape)
df.head()


Raw df shape (before cleaning): (3955, 2)


,report_id,report_text
0,401,Mediastinal contours are normal. Blunting of t...
1,3738,The heart size and mediastinal contours appear...
2,3099,"Lungs are clear without focal consolidation, e..."
3,862,The parenchymal scar in the left lower lobe is...
4,3834,There is an S-shaped scoliosis of the thoracic...


In [5]:

# 5. CLEAN TEXT AND CREATE WEAK NODULE LABEL

# Drop rows where report_text is empty or missing
df["report_text"] = df["report_text"].fillna("").str.strip()
df = df[df["report_text"] != ""].reset_index(drop=True)

print("After dropping empty report_text rows:", df.shape)

# Weak labeling strategy:
# Mark a report as nodule-positive if it contains at least one
# nodule-related keyword. This is a common weak supervision approach
# when radiologist ground-truth labels are not available.
NUDULE_KEYWORDS = [
    "nodule",
    "nodular",
    "pulmonary nodule",
    "lung nodule",
    "mass",
    "opacity"
]

def weak_label_nodule(text):
    """
    Return 1 if the report text contains any nodule-related keyword,
    otherwise return 0.
    """
    if not isinstance(text, str):
        return 0
    t = text.lower()
    return int(any(kw in t for kw in NUDULE_KEYWORDS))

df["label_nodule"] = df["report_text"].apply(weak_label_nodule)

print("Label distribution (weak labels):")
print(df["label_nodule"].value_counts())
df.head()


After dropping empty report_text rows: (3927, 2)
Label distribution (weak labels):
label_nodule
0    3170
1     757
Name: count, dtype: int64


,report_id,report_text,label_nodule
0,401,Mediastinal contours are normal. Blunting of t...,0
1,3738,The heart size and mediastinal contours appear...,0
2,3099,"Lungs are clear without focal consolidation, e...",0
3,862,The parenchymal scar in the left lower lobe is...,1
4,3834,There is an S-shaped scoliosis of the thoracic...,0


In [6]:
# 6. SAVE PROCESSED DATASET FOR REPRODUCIBILITY

# Save the processed dataset so the pipeline can be reproduced
df.to_csv("reports.csv", index=False)
print("Saved reports.csv with shape:", df.shape)
df.head()

# (Optional) Example of how to reload the file:
# df = pd.read_csv("reports.csv")


Saved reports.csv with shape: (3927, 3)


,report_id,report_text,label_nodule
0,401,Mediastinal contours are normal. Blunting of t...,0
1,3738,The heart size and mediastinal contours appear...,0
2,3099,"Lungs are clear without focal consolidation, e...",0
3,862,The parenchymal scar in the left lower lobe is...,1
4,3834,There is an S-shaped scoliosis of the thoracic...,0


In [7]:
# 7. RELOAD DATA AND PREPARE FOR NLP MODEL

# Reload the processed dataset from CSV (simulates a fresh session)
df = pd.read_csv("reports.csv")
print("Reloaded df shape:", df.shape)

def clean_text(text):
    """
    Simple text normalization:
    - lowercasing
    - collapse multiple spaces
    """
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Add a cleaned version of the report text for modeling
df["report_clean"] = df["report_text"].apply(clean_text)
df[["report_id", "report_clean"]].head()


Reloaded df shape: (3927, 3)


,report_id,report_clean
0,401,mediastinal contours are normal. blunting of t...
1,3738,the heart size and mediastinal contours appear...
2,3099,"lungs are clear without focal consolidation, e..."
3,862,the parenchymal scar in the left lower lobe is...
4,3834,there is an s-shaped scoliosis of the thoracic...


In [8]:
# 8. TRAIN/TEST SPLIT

X = df["report_clean"]
y = df["label_nodule"]

print("Label counts:")
print(y.value_counts())

# Stratified split to preserve class balance in train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

len(X_train), len(X_test)


Label counts:
label_nodule
0    3170
1     757
Name: count, dtype: int64


(3141, 786)

In [9]:
# 9. TF–IDF VECTORIZATION

# Convert the cleaned text into TF–IDF features (unigrams + bigrams)
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3  # ignore very rare terms; adjust if dataset is small
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_vec.shape, X_test_vec.shape


((3141, 5112), (786, 5112))

In [10]:
# 10. TRAIN LOGISTIC REGRESSION CLASSIFIER

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_vec, y_train)

# Evaluate on held-out test set
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      1.00      0.96       634
           1       1.00      0.66      0.79       152

    accuracy                           0.93       786
   macro avg       0.96      0.83      0.88       786
weighted avg       0.94      0.93      0.93       786



In [11]:
# 11. EXTRACT NODULE SIZE FROM TEXT

# Regex to capture patterns like "6 mm" or "0.8 cm"
size_pattern = re.compile(r"(\d+(\.\d+)?)\s*(mm|millimeter|cm|centimeter)")

def extract_nodule_size_mm(text):
    """
    Extract a single nodule size from the report text and convert to mm.
    Returns:
        float (size in mm) or None if no size is found.
    """
    if not isinstance(text, str):
        return None

    match = size_pattern.search(text.lower())
    if match:
        value = float(match.group(1))
        unit = match.group(3)
        # Normalize to millimeters
        if unit.startswith("cm"):
            return value * 10.0
        return value
    return None

# Quick sanity check
examples = [
    "There is a 6 mm pulmonary nodule in the right upper lobe.",
    "An 0.8 cm nodule is present in the left lower lobe.",
    "Tiny nodule is seen, size not specified."
]

for t in examples:
    print(t, "->", extract_nodule_size_mm(t))


There is a 6 mm pulmonary nodule in the right upper lobe. -> 6.0
An 0.8 cm nodule is present in the left lower lobe. -> 8.0
Tiny nodule is seen, size not specified. -> None


In [13]:
# 12. SIMPLIFIED FLEISCHNER-LIKE RULE ENGINE
#   (EDUCATIONAL ONLY – NOT FOR CLINICAL USE)

def fleischner_like_recommendation(size_mm, risk="low"):
    """
    Educational, simplified version of guideline-style follow-up rules
    for incidental pulmonary nodules.

    Args:
        size_mm (float or None): nodule size in mm if detected.
        risk (str): "low" or "high" clinical risk category.

    Returns:
        str: Human-readable follow-up recommendation.
    """
    if size_mm is None:
        return (
            "Nodule detected, but size unknown. "
            "Recommend reviewing prior imaging and clinical risk factors."
        )

    if size_mm < 6:
        if risk == "low":
            return "Nodule <6 mm (low risk): No routine follow-up."
        else:
            return "Nodule <6 mm (high risk): Optional CT at 12 months."
    elif 6 <= size_mm <= 8:
        return "Nodule 6–8 mm: Recommend CT at 6–12 months."
    else:
        return "Nodule >8 mm: Consider CT at ~3 months, PET/CT, or tissue sampling."


In [15]:
# 13. FULL CDSS PIPELINE FUNCTION

def run_cdss_on_report(text, risk="low", threshold=0.5):
    """
    Run the end-to-end CDSS on a single free-text radiology report.

    Steps:
      1. Clean the text.
      2. Use the NLP model to estimate probability of a pulmonary nodule.
      3. If positive, attempt to extract nodule size.
      4. Apply simplified Fleischner-like rules to suggest follow-up.

    Args:
        text (str): Original radiology report text.
        risk (str): "low" or "high" clinical risk.
        threshold (float): Probability cutoff for classifying nodule presence.

    Returns:
        dict: Structured CDSS output with keys:
              - nodule_detected (bool)
              - probability (float)
              - size_mm (float or None)
              - recommendation (str)
    """
    clean = clean_text(text)
    X_vec = vectorizer.transform([clean])
    prob = clf.predict_proba(X_vec)[0, 1]

    # If model does not think this is a nodule-containing report
    if prob < threshold:
        return {
            "nodule_detected": False,
            "probability": float(prob),
            "size_mm": None,
            "recommendation": "No pulmonary nodule detected."
        }

    # If nodule is likely present, try to extract size and apply rules
    size = extract_nodule_size_mm(clean)
    rec = fleischner_like_recommendation(size, risk)

    return {
        "nodule_detected": True,
            "probability": float(prob),
        "size_mm": size,
        "recommendation": rec
    }


In [16]:
# 14. DEMONSTRATION ON SAMPLE REPORTS

# Sample a few reports to illustrate how the CDSS behaves
sample_df = df.sample(5, random_state=1)

for _, row in sample_df.iterrows():
    print("=" * 80)
    print("Report ID:", row["report_id"])
    print("Report Text:", row["report_text"][:300], "...\n")  # truncated for display

    result = run_cdss_on_report(row["report_text"], risk="low")
    print("CDSS Output:")
    for k, v in result.items():
        print(f"  {k}: {v}")
    print()


Report ID: 3723
Report Text: Lungs are clear bilaterally. Cardiac and mediastinal silhouettes are normal. Pulmonary vasculature is normal. No pneumothorax or pleural effusion. No acute bony abnormality. No acute cardiopulmonary abnormality. ...

CDSS Output:
  nodule_detected: False
  probability: 0.024840512467023776
  size_mm: None
  recommendation: No pulmonary nodule detected.

Report ID: 3320
Report Text: Chest. Heart size normal lungs are clear. Right knee. Severe osteoarthritis all 3 compartments ...

CDSS Output:
  nodule_detected: False
  probability: 0.08262776360477554
  size_mm: None
  recommendation: No pulmonary nodule detected.

Report ID: 3301
Report Text: The cardiomediastinal silhouette is within normal limits for size and contour. The lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothorax. Osseous structures are within normal limits for patient age.. 1. No acute radiographic cardiopulmonary process ...

CDSS Output:
